In [ ]:
"""
author: Liron Levin

run chaperone X chaperones
+ bunfferoni
"""

import pandas as pd
import numpy as np
from scipy import stats
from rpy2.robjects import pandas2ri

pandas2ri.activate()
from rpy2.robjects.packages import importr
pheatmap = importr('pheatmap')

def FDR(pvalmat):
    stats = importr('stats')
    p=list()
    map(lambda x: p.extend(pvalmat[x]) ,pvalmat.columns)
    p_j=stats.p_adjust(p, method = 'fdr')
    p_ajast=pvalmat.copy()
    p_ajast.loc[:,:]=np.reshape(p_j,(len(pvalmat.columns),len(pvalmat.index))).T
    return p_ajast

def p_val_cutoff(pvalmat,q):
    # the value -1 marks that the cutoff will be the bonferroni value and not a percentile.
    if q == -1:
        return 2.9 * (10 ** -6)
    
    p=list()
    map(lambda x: p.extend(pvalmat[x]) ,pvalmat.columns)
    return np.percentile(np.asanyarray(p)[~np.isnan(p)],q)
 
def Run_corr(tissue,analysis_type,q,Normal='Solid Tissue Normal',Cancer='Primary Tumor',
             sample_size=2000,dist='manhattan', file_path='Results\\'):
    #analysis_type='match' or 'percentile'
    #q=40
    print tissue
    
    # read needed gene data
    Mito_genes = pd.read_table('Mito_ch_genes.tab',sep='\t')
    Mito_ch=pd.read_table('Mito_ch.tab',sep='\t')
    ENSID2Symbol=Mito_genes.set_index('ENSID')['Symbol'].to_dict()

    # build samples data to 
    Files_Data=pd.read_table(tissue+'/MANIFEST.txt',sep='\t',index_col=0)
    Clinical_Data=pd.read_table(tissue+'/clinical.tsv',sep='\t',index_col=0)
    Sample_Data=pd.read_table(tissue+'/gdc_sample_sheet.tsv',sep='\t',index_col=0)
    Files_Data=Files_Data.join(Sample_Data).merge(Clinical_Data,left_on='Case ID',right_on='submitter_id').set_index('Case ID')

    # filtering samples by caterogys if needed
    Data_NORMAL=Files_Data.loc[ (Files_Data['race']!='@white') & ( Files_Data['gender']!='@female') & ( Files_Data['Sample Type']==Normal)  & ( Files_Data['state']=='live')   ,:]
    Data_CANCER=Files_Data.loc[ (Files_Data['race']!='@white') & ( Files_Data['gender']!='@female') & ( Files_Data['Sample Type']==Cancer)  & ( Files_Data['state']=='live')   ,:]

    # if match analysis filter only the samples matched with cancer samples
    if analysis_type=='matched':
        filtered_index=filter(lambda x: x in Data_NORMAL.index, Data_CANCER.index)
        if len(filtered_index)>0:
            Data_CANCER=Data_CANCER.loc[filtered_index,:]
        else:
            Data_CANCER=[]

    # if we have any samples to work with
    if (len(Data_NORMAL)>20)&(len(Data_CANCER)>20):
        count=0

        for F in Data_NORMAL['filename'] :
            if type(F)==str:
                if count==0:
                    TCGA_Data=pd.read_table(tissue+'/'+F,sep='\t',names=['ENSID',tissue+str(count)],index_col=0)
                else:
                    TCGA_Data=TCGA_Data.join( pd.read_table(tissue+'/'+F,sep='\t',names=['ENSID',tissue+str(count)],index_col=0), how='inner')
                if count==sample_size:
                    break
                count=count+1
        print count 
        Normal_sample_size=TCGA_Data.shape[1]
        
        # for every sample, keep data from the needed gene
        TCGA_Mito_genes_filter=filter(lambda x: x.split('.')[0].replace(' ','') in Mito_genes['ENSID'].values ,TCGA_Data.index)
        TCGA_Mito_ch_filter=filter(lambda x: x.split('.')[0].replace(' ','') in Mito_ch['ENSID'].values ,TCGA_Data.index)

        # calculate correlations
        Mito_gene_TCGA_Data=TCGA_Data.loc[list(set(TCGA_Mito_genes_filter).union(TCGA_Mito_ch_filter)),:]
        Mito_gene_TCGA_Data=Mito_gene_TCGA_Data.T.copy()
        coeffmat = Mito_gene_TCGA_Data.corr(method='spearman')
        coeffmat[coeffmat.isnull()]=0
        pvalmat = pd.DataFrame(columns=coeffmat.columns,index=coeffmat.columns,data=stats.spearmanr(Mito_gene_TCGA_Data)[1])
        coeffmat = coeffmat.loc[TCGA_Mito_ch_filter,]
        #coeffmat = coeffmat.drop(TCGA_Mito_ch_filter, axis=1)
        pvalmat=pvalmat.loc[TCGA_Mito_ch_filter,]
        #pvalmat = pvalmat.drop(TCGA_Mito_ch_filter, axis=1)

        Normal_q_pval_cutoff=p_val_cutoff(pvalmat,q)

        pvalmat.columns= map(lambda x: x.split('.')[0] ,pvalmat.columns)
        pvalmat.index= map(lambda x: x.split('.')[0] ,pvalmat.index)
        pvalmat.index=map(lambda x: ENSID2Symbol[x] +'(Normal)' if ENSID2Symbol.has_key(x) else np.nan  ,pvalmat.index)
        pvalmat.columns=map(lambda x: ENSID2Symbol[x] +'(Normal)' if ENSID2Symbol.has_key(x) else np.nan  ,pvalmat.columns)
        
        coeffmat.columns= map(lambda x: x.split('.')[0] ,coeffmat.columns)
        coeffmat.index= map(lambda x: x.split('.')[0] ,coeffmat.index)
        coeffmat.index=map(lambda x: ENSID2Symbol[x] +'(Normal)' if ENSID2Symbol.has_key(x) else np.nan  ,coeffmat.index)
        coeffmat.columns=map(lambda x: ENSID2Symbol[x] +'(Normal)' if ENSID2Symbol.has_key(x) else np.nan  ,coeffmat.columns)
        coeffmat.to_csv(file_path+tissue+'_corr_Normal_ATCG.tab',sep='\t')    

        Normal_coeffmat=coeffmat.copy()
        Normal_pvalmat=pvalmat.copy()

        count=0

        for F in Data_CANCER['filename'] :
            if type(F)==str:
                if count==0:
                    TCGA_Data=pd.read_table(tissue+'/'+F,sep='\t',names=['ENSID',tissue+str(count)],index_col=0)
                else:
                    TCGA_Data=TCGA_Data.join( pd.read_table(tissue+'/'+F,sep='\t',names=['ENSID',tissue+str(count)],index_col=0), how='inner')
                if count==sample_size:
                    break
                count=count+1

        print count
        Cancer_sample_size=TCGA_Data.shape[1]

        TCGA_Mito_genes_filter=filter(lambda x: x.split('.')[0].replace(' ','') in Mito_genes['ENSID'].values ,TCGA_Data.index)
        TCGA_Mito_ch_filter=filter(lambda x: x.split('.')[0].replace(' ','') in Mito_ch['ENSID'].values ,TCGA_Data.index)

        Mito_gene_TCGA_Data=TCGA_Data.loc[list(set(TCGA_Mito_genes_filter).union(TCGA_Mito_ch_filter)),:]
        Mito_gene_TCGA_Data=Mito_gene_TCGA_Data.T.copy()
        coeffmat = Mito_gene_TCGA_Data.corr(method='spearman')
        coeffmat[coeffmat.isnull()]=0
        #pvalmat = corr_pval2(Mito_gene_TCGA_Data)
        pvalmat = pd.DataFrame(columns=coeffmat.columns,index=coeffmat.columns,data=stats.spearmanr(Mito_gene_TCGA_Data)[1])
        coeffmat = coeffmat.loc[TCGA_Mito_ch_filter,]
        #coeffmat = coeffmat.drop(TCGA_Mito_ch_filter, axis=1)
        pvalmat=pvalmat.loc[TCGA_Mito_ch_filter,]
        #pvalmat = pvalmat.drop(TCGA_Mito_ch_filter, axis=1)

        Cancer_q_pval_cutoff=p_val_cutoff(pvalmat,q)

        pvalmat.columns= map(lambda x: x.split('.')[0] ,pvalmat.columns)
        pvalmat.index= map(lambda x: x.split('.')[0] ,pvalmat.index)
        pvalmat.index=map(lambda x: ENSID2Symbol[x] +'(Cancer)' if ENSID2Symbol.has_key(x) else np.nan  ,pvalmat.index)
        pvalmat.columns=map(lambda x: ENSID2Symbol[x] +'(Cancer)' if ENSID2Symbol.has_key(x) else np.nan  ,pvalmat.columns)
        
        coeffmat.columns= map(lambda x: x.split('.')[0] ,coeffmat.columns)
        coeffmat.index= map(lambda x: x.split('.')[0] ,coeffmat.index)
        coeffmat.index=map(lambda x: ENSID2Symbol[x] +'(Cancer)' if ENSID2Symbol.has_key(x) else np.nan  ,coeffmat.index)
        coeffmat.columns=map(lambda x: ENSID2Symbol[x] +'(Cancer)' if ENSID2Symbol.has_key(x) else np.nan  ,coeffmat.columns)
        #coeffmat.to_csv(file_path+tissue+'_corr_CANCER_ATCG.tab',sep='\t')    

        Cancer_coeffmat=coeffmat.copy()   
        Cancer_pvalmat=pvalmat.copy()


        Cancer_coeffmat['name']=Cancer_coeffmat.index
        Normal_coeffmat['name']=Normal_coeffmat.index
        coeffmat=Cancer_coeffmat.merge(Normal_coeffmat,how='outer')
        coeffmat.index=coeffmat['name']
        coeffmat=coeffmat.drop('name',axis=1)

        ###################################################################################

        if analysis_type=='percentile':
            q_Cancer_pvalmat=Cancer_pvalmat>Cancer_q_pval_cutoff
            q_Normal_pvalmat=Normal_pvalmat>Normal_q_pval_cutoff

            q_Cancer_pvalmat['name']=q_Cancer_pvalmat.index
            q_Normal_pvalmat['name']=q_Normal_pvalmat.index
            q_pvalmat=q_Cancer_pvalmat.merge(q_Normal_pvalmat,how='outer')
            q_pvalmat.index=q_pvalmat['name']
            q_pvalmat=q_pvalmat.drop('name',axis=1) 


            q_coeffmat=coeffmat.copy()
            q_coeffmat[q_pvalmat==True]=0

            
            q_coeffmat[q_coeffmat.isnull()]=0
            
            data=pheatmap.pheatmap(pandas2ri.py2ri(q_coeffmat),show_colnames = False,treeheight_row = 100,clustering_method="ward.D",
                              filename=file_path+tissue+'_'+str(q)+'_'+analysis_type+'_Normal'+str(Normal_sample_size)+'_Cancer'+str(Cancer_sample_size)+'.pdf' ,
                              silent =True   ,clustering_distance_rows= dist)        

            row=data[0][2]
            col=data[1][2]
            q_coeffmat.iloc[map(lambda x:x-1,row),map(lambda x:x-1,col)].to_csv(file_path+tissue+'_'+str(q)+'_'+analysis_type+'_Normal'+str(Normal_sample_size)+'_Cancer'+str(Cancer_sample_size)+'_corr_ATCG.tab',sep='\t') 

            Cancer_pvalmat['name']=Cancer_pvalmat.index
            Normal_pvalmat['name']=Normal_pvalmat.index
            pvalmat=Cancer_pvalmat.merge(Normal_pvalmat,how='outer')
            pvalmat.index=pvalmat['name']
            pvalmat=pvalmat.drop('name',axis=1)

            pvalmat.iloc[map(lambda x:x-1,row),map(lambda x:x-1,col)].to_csv(file_path+tissue+'_'+str(q)+'_'+analysis_type+'_Normal'+str(Normal_sample_size)+'_Cancer'+str(Cancer_sample_size)+'_p_val_ATCG.tab',sep='\t') 

        else:
            ##########################################################################################

            Cancer_pvalmat['name']=Cancer_pvalmat.index
            Normal_pvalmat['name']=Normal_pvalmat.index
            pvalmat=Cancer_pvalmat.merge(Normal_pvalmat,how='outer')
            pvalmat.index=pvalmat['name']
            pvalmat=pvalmat.drop('name',axis=1) 

            pvalmat=FDR(pvalmat)
            coeffmat[pvalmat>0.05]=0
            coeffmat[coeffmat.isnull()]=0
            coeffmat=coeffmat.loc[coeffmat.columns,:]

            coeffmat.to_csv(file_path+tissue+'_'+analysis_type+'_corr_ATCG.tab',sep='\t')     
            data=pheatmap.pheatmap(pandas2ri.py2ri(coeffmat),show_colnames = True,
                                   treeheight_row = 100,
                                   cluster_cols =False,
                                   cluster_rows =False,
                                   labels_col = coeffmat.columns,
                                   labels_row = coeffmat.index,
                                   clustering_method="ward.D",
                              filename=file_path+tissue+'_'+analysis_type+'_Normal'+str(Normal_sample_size)+'_Cancer'+str(Cancer_sample_size)+'.pdf' ,
                              main =tissue+'_'+analysis_type ,
                              silent =True   ,clustering_distance_rows= dist)
            

print 'done'

In [ ]:
# here will be ran the correlation analysis thorugh bonffirini values.
tissue_list = ['Breast Invasive Carcinoma',
               'Colon_Adenocarcinoma',
               'Head_and_Neck_Squamous_Cell_Carcinoma',
               'Kidney_Chromophobe',
               'Kidney_Renal_Clear_Cell_Carcinoma',
               'Kidney_Renal_Papillary_Cell_Carcinoma',
               'Liver_Hepatocellular_Carcinoma',
               'Lung_Adenocarcinoma',
               'Lung_Squamous_Cell_Carcinoma',
               'Prostate_Adenocarcinoma',
               'Stomach_Adenocarcinoma',
               'Thyroid_Carcinoma',
               'Uterine_Corpus_Endometrial_Carcinoma']

for tissue_name in tissue_list:
    output_path_bonf = 'Results chapXchap correct bonf\\' + tissue_name + '\\'
    Run_corr(tissue_name,'percentile',-1 ,sample_size=2000,dist='manhattan', file_path=output_path_bonf)
    
    print tissue_name + ' - done.'

print "done"